In [3]:
import numpy as np
from sklearn.model_selection import train_test_split

In [16]:
test_size = 0.3
x=np.load('loaded data/jl full x.npy') 
y=np.load('loaded data/jl full y.npy') 
x_train,x_test,temp_y_train,temp_y_test=train_test_split(np.array(x), y, test_size = test_size, random_state = 9)


In [6]:
x_traincnn = np.expand_dims(x_train, axis=2)
x_testcnn = np.expand_dims(x_test, axis=2)



In [7]:
x_traincnn.shape, x_testcnn.shape


((1176, 180, 1), (504, 180, 1))

In [17]:
y_test,y_train = [],[]
d={}
for i in temp_y_test:
    if i not in d:
        d[i] = len(d)
    y_test.append(d[i])
for i in temp_y_train:
    if i not in d:
        d[i] = len(d)
    y_train.append(d[i])



In [18]:
y_train=np.array(y_train)
y_test=np.array(y_test)

In [21]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv1D(128, 5,padding='same',
                 input_shape=(180,1)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(8))
model.add(Activation('softmax'))
opt = tf.keras.optimizers.RMSprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)


/home/vedant/miniconda3/lib/python3.9/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [22]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 180, 128)          768       
                                                                 
 activation_9 (Activation)   (None, 180, 128)          0         
                                                                 
 dropout_6 (Dropout)         (None, 180, 128)          0         
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 22, 128)          0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, 22, 128)           82048     
                                                                 
 activation_10 (Activation)  (None, 22, 128)           0         
                                                      

In [23]:


model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])



In [24]:
history=model.fit(x_traincnn, y_train, batch_size=16, epochs=1000, validation_data=(x_testcnn, y_test))

Epoch 1/1000


2022-03-04 16:07:24.200290: E tensorflow/stream_executor/cuda/cuda_dnn.cc:371] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2022-03-04 16:07:24.200352: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1120 : UNKNOWN: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_3/conv1d_6/Conv1D
 (defined at /home/vedant/miniconda3/lib/python3.9/site-packages/keras/layers/convolutional.py:231)
]] [Op:__inference_train_function_2439]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_3/conv1d_6/Conv1D:
In[0] sequential_3/conv1d_6/Conv1D/ExpandDims:	
In[1] sequential_3/conv1d_6/Conv1D/ExpandDims_1:

Operation defined at: (most recent call last)
>>>   File "/home/vedant/miniconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/home/vedant/miniconda3/lib/python3.9/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/home/vedant/.local/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
>>>    